# 1. Node Classification

In [1]:
import snap
import numpy as np

## 1.1 Relational Classification

In [2]:
# Generating Graph Structure
G = snap.TUNGraph.New()
# Add Nodes
for i in range(10):
    G.AddNode(i+1)
# Add Edges
Edges = [(1, 2), (1, 3), (2, 3), (2, 4), (3, 6), (4, 7), (4, 8), 
         (5, 6), (5, 8), (5, 9), (6, 9), (6, 10), (7, 8), (8, 9), (9, 10)]
for src, dst in Edges:
    G.AddEdge(src, dst)

In [3]:
# Generating label and flag vectors: 
# labels: probability that the node is positive
# flags: whether the label is ground truth (True) or prediction (False):
labels = [None, 0.5, 0.5, 1, 0.5, 1, 0.5, 0.5, 0, 0.5, 0]
flags = [None, False, False, True, False, True, False, False, True, False, True]

In [4]:
# Iterate
maxIter = 2
for epoch in range(maxIter):
    for node in G.Nodes():
        node_id = node.GetId()
        if flags[node_id]:  # is ground truth
            continue  # do not update
        
        neighbors = [node.GetNbrNId(i) for i in range(node.GetDeg())]
        neighbors_labels = [labels[n] for n in neighbors]
        labels[node_id] = np.mean(neighbors_labels)

(i) After the second iteration, $P(Y_i = +)$ for $i = 2, 4, 6$

In [5]:
print(labels[2], labels[4], labels[6])

0.7638888888888888 0.32407407407407407 0.6015625


(ii) final state

In [6]:
# Keep Iterating for another 10 epochs
# Iterate
maxIter = 10
for epoch in range(maxIter):
    for node in G.Nodes():
        node_id = node.GetId()
        if flags[node_id]:  # is ground truth
            continue  # do not update
        
        neighbors = [node.GetNbrNId(i) for i in range(node.GetDeg())]
        neighbors_labels = [labels[n] for n in neighbors]
        labels[node_id] = np.mean(neighbors_labels)

In [7]:
labels[1:]

[0.857146612937504,
 0.7142884876278369,
 1,
 0.2857159708669467,
 1,
 0.6000000000000014,
 0.14285798543347336,
 0,
 0.40000000000000036,
 0]

Node 1, 2, 3, 5, 6 will be positive,  
Node 4, 7, 8, 9, 10 will be negative.  

## 1.2 Belief Propagation

(i)  
$$
\begin{aligned}
m_{32}(x_2) &= \sum_{x_3} \phi(x_3) \psi_{32}(x_3, x_2) \\
m_{42}(x_2) &= \sum_{x_4} \phi(x_4) \psi_{42}(x_4, x_2) \\
m_{21}(x_1) &= \sum_{x_2} \phi(x_2) \psi_{21}(x_2, x_1) m_{32}(x_2) m_{42}(x_2) \\
&= \sum_{x_2} \phi(x_2) \psi_{21}(x_2, x_1) \sum_{x_3} \phi(x_3) \psi_{32}(x_3, x_2) \sum_{x_4} \phi(x_4) \psi_{42}(x_4, x_2) \\
b_1(x_1) &= \frac{1}{Z} \phi_1(x_1) m_{21}(x_1) \\
&= \frac{1}{Z} \phi_1(x_1)\sum_{x_2} \phi(x_2) \psi_{21}(x_2, x_1) \sum_{x_3} \phi(x_3) \psi_{32}(x_3, x_2) \sum_{x_4} \phi(x_4) \psi_{42}(x_4, x_2)
\end{aligned}
$$

(ii)
$$
\begin{aligned}
p(x_1 | y_1, y_2, y_3, y_4) &= \sum_{x_2}\sum_{x_3}\sum_{x_4}p(x_1, x_2, x_3, x_4 | y_1, y_2, y_3, y_4) \\
&= \sum_{x_2}\sum_{x_3}\sum_{x_4} \frac{1}{Z} \phi(x_1)\phi(x_2)\phi(x_3)\phi(x_4)
\psi_{21}(x_2, x_1)\psi_{32}(x_3, x_2)\psi_{42}(x_4, x_2) \\
&= \frac{1}{Z} \phi(x_1) \sum_{x_2}\sum_{x_3}\sum_{x_4}\phi(x_2)\phi(x_3)\phi(x_4)
\psi_{21}(x_2, x_1)\psi_{32}(x_3, x_2)\psi_{42}(x_4, x_2) \\
&= \frac{1}{Z} \phi(x_1) \sum_{x_2} \phi(x_2) \psi_{21}(x_2, x_1) \sum_{x_3}\sum_{x_4}\phi(x_3)\phi(x_4)
\psi_{32}(x_3, x_2)\psi_{42}(x_4, x_2) \\
&= \frac{1}{Z} \phi(x_1) \sum_{x_2} \phi(x_2) \psi_{21}(x_2, x_1) \sum_{x_3}\phi(x_3)\psi_{32}(x_3, x_2)
\sum_{x_4}\phi(x_4)
\psi_{42}(x_4, x_2) \\
\end{aligned}
$$

(iii)  
Ground truth is $y_2 = 0$ and $y_4 = 1$.  
As both $\phi_2$ and $\phi_4$ are strong, we should have $x_2 \approx 0$ and $x_4 \approx 1$. (Here '$\approx$' means 'having higher probability to be equal to')  
Compare to $\psi_{12}$ and $\psi_{34}$, $\psi_{23}$ and $\psi_{35}$ are more strongly dependent. So $x_3 \approx 1$ and $x_5 \approx 0$.  
Consider the weakest $\psi_{12}$ and $\psi_{34}$, they show slight preference to have same label on the both side. So we have $x_1 \approx 1$, but the probabilities to be positive will be close to 0.5. 

In [8]:
# Ground Truth: array([P(y = 0), P(y = 1)])
y_2 = np.array([1, 0])
y_4 = np.array([0, 1])
psi_12 = np.array([[1, 0.9], [0.9, 1]])
psi_34 = np.array([[1, 0.9], [0.9, 1]])
psi_23 = np.array([[0.1, 1], [1, 0.1]])
psi_35 = np.array([[0.1, 1], [1, 0.1]])
phi_2 = np.array([[1, 0.1], [0.1, 1]])
phi_4 = np.array([[1, 0.1], [0.1, 1]])

In [9]:
# Message Passing: right to left
m_y4_x4 = (y_4.reshape((1, -1)) * phi_4).sum(axis=1)
m_x4_x3 = (m_y4_x4.reshape((1, -1)) * psi_34).sum(axis=1)
m_x5_x3 = psi_35.sum(axis=1)
m_x3_x2 = ((m_x4_x3 * m_x5_x3).reshape((1, -1)) * psi_23).sum(axis=1)
m_y2_x2 = (y_2.reshape((1, -1)) * phi_2).sum(axis=1)
m_x2_x1 = ((m_x3_x2 * m_y2_x2).reshape((1, -1)) * psi_12).sum(axis=1)

# Message Passing: left to right
m_x1_x2 = psi_12.sum(axis=0)
m_x2_x3 = ((m_x1_x2 * m_y2_x2).reshape((-1, 1)) * psi_23).sum(axis=0)
m_x3_x4 = ((m_x2_x3 * m_x5_x3).reshape((-1, 1)) * psi_34).sum(axis=0)
m_x3_x5 = ((m_x2_x3 * m_x4_x3).reshape((-1, 1)) * psi_35).sum(axis=0)

Note that we compute only $m_{y_2x_2}$ and $m_{y_4x_4}$, not $m_{x_2y_2}$ and $m_{x_4y_4}$, because $y_2$ and $y_4$ are ground truth. Any message passing to them will not change their distribution. 

In [10]:
# Belief
b_1 = m_x2_x1
b_1 = b_1 / b_1.sum()
b_2 = m_x1_x2 * m_x3_x2 * m_y2_x2
b_2 = b_2 / b_2.sum()
b_3 = m_x2_x3 * m_x4_x3 * m_x5_x3
b_3 = b_3 / b_3.sum()
b_4 = m_x3_x4 * m_y4_x4
b_4 = b_4 / b_4.sum()
b_5 = m_x3_x5
b_5 = b_5 / b_5.sum()

In [11]:
print('       p(x=0)     p(x=1)')
print('b_1: %s\nb_2: %s\nb_3: %s\nb_4: %s\nb_5: %s' % (b_1, b_2, b_3, b_4, b_5))

       p(x=0)     p(x=1)
b_1: [0.52182902 0.47817098]
b_2: [0.91475133 0.08524867]
b_3: [0.15373972 0.84626028]
b_4: [0.08524867 0.91475133]
b_5: [0.78330387 0.21669613]


The prediction is similar to our expectation. 